## Setup

In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime as dt
import sqlite3

In [29]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

--2024-08-09 06:24:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/csv]
Saving to: ‘exchange_rate.csv.1’

exchange_rate.csv.1 100%[===================>]      45  --.-KB/s    in 0s      

2024-08-09 06:24:38 (19.1 MB/s) - ‘exchange_rate.csv.1’ saved [45/45]



In [30]:
data_url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

extracted_table_attritbutes = ["Name", "MC_USD_Billion"]
final_table_attributes = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]
output_file_path = "./Largest_banks_data.csv"

database_name = "Banks.db"

table_name = "Largest_banks"
log_filename = "code_log.txt"

## Task 1

In [31]:
def log_progress(message):

    ''' This function logs the mentioned message of a given stage of the
        code execution to a log file. Function returns nothing'''

    now = dt.datetime.utcnow()
    timestamp = now.replace(microsecond=0)
    with open(log_filename, "a") as f:
      f.write(f'Timestamp: {timestamp} -----  {message}\n')

## Task 2

In [32]:
def extract(url, attribute_names):
  ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''


  data = requests.get(url).text
  soup = BeautifulSoup(data, "html.parser")
  tables = soup.find_all("tbody")
  trs = tables[0].find_all("tr")

  names= []
  mc = []

  for tr in trs[1:]:
      tds = tr.find_all("td")
      if len(tds) > 0:
        names.append(tds[1].find_all("a")[-1]["title"])
        mc.append(float(tds[2].contents[0][:-1]))

  df = pd.DataFrame(zip(names, mc), columns=attribute_names)
  return df

## Task 3

In [33]:
def transform(dataframe, csv_filename):
  ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

  exchange_df = pd.read_csv(csv_filename)

  for row in exchange_df.iterrows():
    cur = row[1].Currency
    rate = row[1].Rate
    dataframe[f"MC_{cur}_Billion"] = round(rate * dataframe["MC_USD_Billion"], 2)

  return dataframe

## Task 4

In [34]:
def load_to_csv(dataframe, output_file):
  ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

  dataframe.to_csv(output_file, index=False)

## Task 5

In [35]:
def load_to_db(dataframe, db_name, table):
  ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''


  conn = sqlite3.connect(db_name)
  dataframe.to_sql(table, conn, if_exists="replace")
  conn.close()

## Task 6

In [40]:
def run_query(query, db_name):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    conn = sqlite3.connect(db_name)
    results = pd.read_sql(query, conn)
    conn.close()
    if "index" in results.columns:
        results.drop(["index"], axis=1, inplace=True)
    return results

## Task 7


In [37]:
def check_log(filename):
  with open(filename, "r") as f:
    for line in f.readlines():
      print(line)

# Execution

In [43]:
log_progress("Process Started")

# Extraction
log_progress("Extraction Starting")
df = extract(data_url, extracted_table_attritbutes)
print(df)

log_progress("Extraction Completed")

# Transformation
log_progress("Transformation Starting")

df1 = transform(df, "exchange_rate.csv")
df1 = df1[final_table_attributes]

print(df1)

log_progress("Transformation Completed")

# saving to CSV
log_progress("Loading data to csv file")

load_to_csv(df1, output_file_path)

log_progress("Data loaded to csv file")

# Saving to database
log_progress("Loading data to database")

load_to_db(df1, database_name, table_name)

log_progress("Data loaded to database")

# Run query
log_progress("Running query")

query1 = "SELECT * FROM Largest_banks"
result = run_query(query1, database_name)
print(f"Query 1: {query1}\n{result}")

query2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
result = run_query(query2, database_name)
print(f"Query 2: {query2}\n{result}")


query3 = "SELECT Name from Largest_banks LIMIT 5"
result = run_query(query3, database_name)
print(f"Query 3: {query3}\n{result}")



log_progress("Query completed")

# Check logs()
print("\n\n")
check_log(log_filename)

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of Ch